## <b><font color='darkblue'>Preface</font></b>
([course link](https://learn.deeplearning.ai/courses/functions-tools-agents-langchain/lesson/7/conversational-agent)) <b><font size='3ptx'>The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order...([more](https://python.langchain.com/v0.1/docs/modules/agents/))</font></b>

Let's summarize what we have learned so far:
* **Agents**:
  * are a combination of LLMs and code
  * LLMs reason about what steps to take and call for actions
* **Agent loop**
  * Choose a tool to use
  * Observe the output of the tool
  * Repeat until a stopping condition is met
* **Stopping conditions can be**:
  * Hardcoded rules
  * LLM determined

### <b><font color='darkgreen'>Agenda</font></b>
In this lab you will
* Build some tools
* Write your own agent loop using LCEL
* Utilize agent_executor which:
  * Implements the agent loop
  * Adds error handling, early stopping, tracing, etc.

In [1]:
!pip freeze | grep -P '(openai|langchain)'

langchain==0.2.12
langchain-anthropic==0.1.15
langchain-community==0.2.6
langchain-core==0.2.28
langchain-experimental==0.0.62
langchain-google-genai==1.0.6
langchain-groq==0.1.3
langchain-openai==0.1.9
langchain-text-splitters==0.2.0
langchainhub==0.1.14
openai==1.28.1


In [29]:
import datetime
import json
import os
import openai
import re
import requests
import httpx
import os
from dotenv import load_dotenv, find_dotenv
from typing import List, Optional
from pydantic import BaseModel, Field, TypeAdapter

import openai
from openai import OpenAI
from langchain.agents import tool
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.tools.render import format_tool_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import AgentExecutor

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## <b><font color='darkblue'>Conversational agent</font></b>
<b><font size='3ptx'>Here we are going to built an agent which can execute the tools provided ot it.</font></b>

### <b><font color='darkgreen'>Built Tools</font></b>
Let's prepare some tools for agent to play with:

#### <b>Tool to get temperature by given latitude and longitude</b>

In [3]:
# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

In [4]:
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

#### <b>Tool to query Wikipedia</b>

In [5]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

So now we have all the needed tools:

In [6]:
tools = [get_current_temperature, search_wikipedia]

### <b><font color='darkgreen'>Phase 1: Agent could decide which tool to execute</font></b>
Our first step is to check if the agent could select the proper tool for execution:

In [7]:
functions = [convert_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

Let's ask a few questions to verify how angent perform:

In [8]:
result = chain.invoke({"input": "What is the weather is sf?"})

In [9]:
# Agent select `get_current_temperature` tool to execute which is expected:
result.tool

'get_current_temperature'

In [10]:
# The input arguments to the tool are also correct:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

### <b><font color='darkgreen'>Phase 2: Agent with `MessagesPlaceholder`</font></b>
<b><font size='3ptx'>LangChain also provides MessagesPlaceholder, which gives you full control of what messages to be rendered during formatting ([more](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/#messagesplaceholder)). </font></b>

This can be useful when you are uncertain of what role you should be using for your message prompt templates or when you wish to insert a list of messages during formatting. Let's use an example to explain the usage of it:

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [12]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [13]:
result1 = chain.invoke({
    "input": "What is the weather is sf?",
    "agent_scratchpad": []
})

In [14]:
result1.tool

'get_current_temperature'

In [15]:
observation = get_current_temperature(result1.tool_input)

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [16]:
observation

'The current temperature is 12.3°C'

In [17]:
type(result1)

langchain_core.agents.AgentActionMessageLog

In [18]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-cda1a598-385e-4850-b193-77ccb0221b2a-0', usage_metadata={'input_tokens': 112, 'output_tokens': 25, 'total_tokens': 137})]

In [19]:
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-cda1a598-385e-4850-b193-77ccb0221b2a-0', usage_metadata={'input_tokens': 112, 'output_tokens': 25, 'total_tokens': 137}),
 FunctionMessage(content='The current temperature is 12.3°C', name='get_current_temperature')]

In [20]:
result2 = chain.invoke({
    "input": "what is the weather is sf?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation), ])
})

In [21]:
result2

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 12.3°C.'}, log='The current temperature in San Francisco is 12.3°C.')

Placeholder `agent_scratchpad` is used to keep the processing result by agent. [**`AgentFinish`**](https://python.langchain.com/v0.1/docs/modules/agents/concepts/#agentfinish) represents the final result from an agent, when it is ready to return to the user. It contains a return_values key-value mapping, which contains the final agent output. **Usually, this contains an `output` key containing a string that is the agent's response**.

### <b><font color='darkgreen'>Phase 3: Agent with a hand-made loop to execute tool and return `AgentFinish`</font></b>
LangChain has several abstractions to make working with agents easy:
* **AgentAction**: This is a dataclass that represents the action an agent should take. It has a `tool` property (<font color='brown'>which is the name of the tool that should be invoked</font>) and a `tool_input` property (<font color='brown'>the input to that tool</font>)
* **AgentFinish**: This represents the final result from an agent, when it is ready to return to the user. It contains a return_values key-value mapping, which contains the final agent output. Usually, this contains an `output` key containing a string that is the agent's response.
* **Intermediate Steps**: These represent previous agent actions and corresponding outputs from this CURRENT agent run. These are important to pass to future iteration so the agent knows what work it has already done. This is typed as a `List[Tuple[AgentAction, Any]]`. Note that observation is currently left as type `Any` to be maximally flexible. In practice, this is often a string.

In [22]:
from langchain.schema.agent import AgentFinish

In [23]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [24]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [25]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [26]:
run_agent("What is the weather is Taipei?")

AgentFinish(return_values={'output': 'The current temperature in Taipei is 31.4°C.'}, log='The current temperature in Taipei is 31.4°C.')

In [27]:
run_agent("What is Bluetooth?")

AgentFinish(return_values={'output': 'Bluetooth is a short-range wireless technology standard used for exchanging data between fixed and mobile devices over short distances. It operates in the 2.4 GHz frequency range and is commonly used for connecting devices like cell phones, music players, and wireless headphones. The technology is managed by the Bluetooth Special Interest Group (SIG) and has over 35,000 member companies. Bluetooth Low Energy (BLE) is a variant of Bluetooth that offers reduced power consumption and cost for specific applications in industries like healthcare and fitness.'}, log='Bluetooth is a short-range wireless technology standard used for exchanging data between fixed and mobile devices over short distances. It operates in the 2.4 GHz frequency range and is commonly used for connecting devices like cell phones, music players, and wireless headphones. The technology is managed by the Bluetooth Special Interest Group (SIG) and has over 35,000 member companies. Blu

In [28]:
# Agent should know this question doesn't require to execute any tool:
run_agent("Hi!")

AgentFinish(return_values={'output': 'Well, hello there! How can I assist you today?'}, log='Well, hello there! How can I assist you today?')

### <b><font color='darkgreen'>Phase 4: `AgentExecutor`</font></b>
<b><font size='3ptx'>AgentExecutor was essentially a runtime for agents. It was a great place to get started...([more](https://python.langchain.com/v0.1/docs/modules/agents/concepts/#agentexecutor))</font></b>

The agent executor is the runtime for an agent. This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats. 

In [30]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [33]:
agent_executor.invoke({"input": "What is Bluetooth?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Bluetooth'}`


Page: Bluetooth
Summary: Bluetooth is a short-range wireless technology standard that is used for exchanging data between fixed and mobile devices over short distances and building personal area networks (PANs). In the most widely used mode, transmission power is limited to 2.5 milliwatts, giving it a very short range of up to 10 metres (33 ft). It employs UHF radio waves in the ISM bands, from 2.402 GHz to 2.48 GHz. It is mainly used as an alternative to wired connections to exchange files between nearby portable devices and connect cell phones and music players with wireless headphones.
Bluetooth is managed by the Bluetooth Special Interest Group (SIG), which has more than 35,000 member companies in the areas of telecommunication, computing, networking, and consumer electronics. The IEEE standardized Bluetooth as IEEE 802.15.1 but no longer maintains the standard. The Bluetooth SIG o

{'input': 'What is Bluetooth?',
 'output': 'Bluetooth is a short-range wireless technology standard used for exchanging data between fixed and mobile devices over short distances. It operates in the 2.4 GHz frequency range and is commonly used for connecting devices like cell phones, music players, and wireless headphones. The technology is managed by the Bluetooth Special Interest Group (SIG) and has over 35,000 member companies. Bluetooth Low Energy (BLE) is a variant of Bluetooth designed for applications in healthcare, fitness, security, and more, with reduced power consumption and cost.'}

In default, agent doesn't have memory:

In [34]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [35]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
I'm sorry, I don't have access to your personal information like your name. How can I assist you today?

> Finished chain.


{'input': 'what is my name',
 'output': "I'm sorry, I don't have access to your personal information like your name. How can I assist you today?"}

### <b><font color='darkgreen'>Phase 4: `AgentExecutor` with memory</font></b>

In [37]:
from langchain.memory import ConversationBufferMemory

In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [38]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [40]:
# `return_messages=True` is necessary for the chain to store the chat history in `chat_history` in the chain.
memory = ConversationBufferMemory(return_messages=True ,memory_key="chat_history")

In [41]:
agent_executor = AgentExecutor(
    agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [42]:
agent_executor.invoke({"input": "My name is John."})



> Entering new AgentExecutor chain...
Nice to meet you, John! How can I assist you today?

> Finished chain.


{'input': 'My name is John.',
 'chat_history': [HumanMessage(content='My name is John.'),
  AIMessage(content='Nice to meet you, John! How can I assist you today?')],
 'output': 'Nice to meet you, John! How can I assist you today?'}

In [43]:
agent_executor.invoke({"input": "What is my name?"})



> Entering new AgentExecutor chain...
Your name is John.

> Finished chain.


{'input': 'What is my name?',
 'chat_history': [HumanMessage(content='My name is John.'),
  AIMessage(content='Nice to meet you, John! How can I assist you today?'),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is John.')],
 'output': 'Your name is John.'}

In [44]:
agent_executor.invoke({"input": "What's the weather in Tokyo?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 35.6895, 'longitude': 139.6917}`


The current temperature is 33.2°CThe current temperature in Tokyo is 33.2°C. It's quite warm there!

> Finished chain.


{'input': "What's the weather in Tokyo?",
 'chat_history': [HumanMessage(content='My name is John.'),
  AIMessage(content='Nice to meet you, John! How can I assist you today?'),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is John.'),
  HumanMessage(content="What's the weather in Tokyo?"),
  AIMessage(content="The current temperature in Tokyo is 33.2°C. It's quite warm there!")],
 'output': "The current temperature in Tokyo is 33.2°C. It's quite warm there!"}

### <b><font color='darkgreen'>Create a Chatbot</font></b>

In [45]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [46]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [50]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [convert_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [52]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=Str, defer_load=False, height=400, loading_indicator=True)
            [3] Divider()